<a href="https://colab.research.google.com/github/tanidayasuo/TrendBFfactor/blob/main/TrendBFfactor_5a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

input.xlsxを入力にして、output.xlsxを出力するプログラムを作成。

１．input.xlsxの「クラス分け」シートと「配合表」シートを入力にして、output.xlsxの「クラス別配合表」を作成する。

　配合表の行には、時制ごとにTrendFactorが並び、列にはBFと呼ばれる変数がそのBFの強さの値をもって並んでいる。

　この表は、TrendFactorがクラス内で時制ごとにソートされている（本来性_、現状_、志向性_）。

２．次にoutput.xlsxの「クラス別配合表」から「クラス別配合時制別合計表」を作成する。

　クラスごとに本来性、現状、志向性の値の合計を出力する。

３．次にoutput.xlsxの「クラス別配合時制別合計表」から、クラス別時制別正規化配合表を作成する。

　BF合計には、クラス別時制のBF合計の平方根が入る。

　その右のBF変数の値は、元のBF変数の値をBF合計（の平方根）で割った値を入れる。

４.「クラス別時制別正規化配合表」から以下の処理を経て、サンプルにあるような「クラス別BFタイプ」を出力する。

10%未満は「0」
classごとに以下のFunctionによって、タイプ判定をする。

FunctionType( BFi(本来性）,BFi(現状),BFi(志向性) ):

BFi(本来性), BFi(現状), BFi(志向性)の値を比較して、

後続の条件により、指定のタイプを出力する

但し、

BFi：表頭の変数（サブライム、帰属回帰など）

i：クラスごとに1～10

A=Bの条件：A,BをBFi(？）としたとき、A=Bとする条件は、

    |A-B|＜0.1 or max(A,B)/min(A,B) < 1.5

　　とする。

タイプの判定条件は、上から評価

・BFi(本来性), BFi(現状), BFi(志向性)がすべてゼロならば、"none"

・BFi(本来性)がゼロではなく、BFi(現状), BFi(志向性)が共にゼロならば、"d0-1"

・BFi(志向性)がゼロではなく、BFi(本来性), BFi(現状)が共にゼロならば、"d0-2"

・BFi(現状)がゼロではなく、BFi(本来性), BFi(志向性)が共にゼロならば、"d0-3"

・BFi(現状)がゼロでかつ、BFi(本来性)＝BFi(志向性)ならば、"d1-eq"

・BFi(現状)がゼロでかつ、BFi(本来性)＞BFi(志向性)ならば、"d1->"

・BFi(現状)がゼロでかつ、BFi(本来性)＜BFi(志向性)ならば、"d1-<"

・BFi(本来性)がゼロでかつ、BFi(現状),BFi(志向性)共にゼロでないならば、"d2"

・BFi(志向性)がゼロでかつ、BFi(現状),BFi(本来性)共にゼロでないならば、"d3"

・そうでない、すなわち、BFi(本来性), BFi(現状), BFi(志向性)がすべてゼロではないならば、

BFi(本来性)＝BFi(志向性)でかつ、BFi(現状)＝BFi(本来性)または、BFi(現状)＝BFi(志向性)ならば、"s1"

  BFi(本来性)＝BFi(志向性)でかつ、BFi(現状)＞BFi(本来性)または、BFi(現状)＞BFi(志向性)ならば、"s1"
  
  BFi(本来性)＜BFi(志向性)ならば、"d2"
  
  BFi(本来性)＞BFi(志向性)ならば、"d3"

In [15]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import Font, Border
from google.colab import files

# **1. エクセルファイルをアップロード**
uploaded = files.upload()

# ファイル名を取得
input_file_path = list(uploaded.keys())[0]
output_file_path = "output.xlsx"

# **2. Excelファイルを処理**
xls = pd.ExcelFile(input_file_path, engine="openpyxl")

# シートを読み込み
df_classification = pd.read_excel(xls, sheet_name="クラス分け")
df_mixture = pd.read_excel(xls, sheet_name="配合表")

# 列名の調整（「TrendFactor」列を統一）
df_mixture.rename(columns={'Unnamed: 0': 'TrendFactor'}, inplace=True)

# 「クラス分け」情報と結合
df_classified_mixture = df_classification.merge(df_mixture, on="TrendFactor", how="left")

# 時制の順序を定義
time_order = ["本来性", "現状", "志向性"]
df_classified_mixture["time_category"] = df_classified_mixture["TrendFactor"].str.extract(f"({'|'.join(time_order)})")[0]

# クラスごとにTrendFactorを時制順にソート
df_classified_mixture["time_category"] = pd.Categorical(df_classified_mixture["time_category"], categories=time_order, ordered=True)
df_classified_mixture = df_classified_mixture.sort_values(by=["class", "time_category", "TrendFactor"])

# **3. 各表の作成**
# クラス別配合表
df_classified_mixture.set_index(["class", "TrendFactor"], inplace=True)
df_classified_mixture = df_classified_mixture.reset_index()

# クラス別配合時制別合計表
df_class_time_sum = df_classified_mixture.groupby(["class", "time_category"]).sum(numeric_only=True).reset_index()

# **時制の順序を正しく並べる**
df_class_time_sum["time_category"] = pd.Categorical(df_class_time_sum["time_category"], categories=time_order, ordered=True)
df_class_time_sum = df_class_time_sum.sort_values(by=["class", "time_category"])

# **BF合計を「合計の平方根」に変更**
numeric_columns = df_class_time_sum.select_dtypes(include=[np.number]).columns
df_class_time_sum["BF合計"] = np.sqrt(df_class_time_sum[numeric_columns].sum(axis=1))

# クラス別時制別正規化配合表の作成
df_class_time_normalized = df_class_time_sum.copy()
df_class_time_normalized.drop(columns=["BF合計"], inplace=True)
df_class_time_normalized[numeric_columns] = df_class_time_sum[numeric_columns].div(df_class_time_sum["BF合計"], axis=0)

# **時制の順序を正しく並べる**
df_class_time_normalized = df_class_time_normalized.sort_values(by=["class", "time_category"])

# **4. クラス別BFタイプの判定**
df_class_time_normalized[numeric_columns] = df_class_time_normalized[numeric_columns].applymap(lambda x: 0 if x < 0.1 else x)

# **BFi(本来性)＝BFi(志向性) の判定関数**
def is_equal(a, b):
    return abs(a - b) < 0.1 or (max(a, b) / min(a, b) < 1.5 if min(a, b) > 0 else False)

# **BFタイプ判定関数**
def determine_bf_type(bf_h, bf_c, bf_s):
    if bf_h == 0 and bf_c == 0 and bf_s == 0:
        return "none"
    if bf_h != 0 and bf_c == 0 and bf_s == 0:
        return "d0-1"
    if bf_s != 0 and bf_h == 0 and bf_c == 0:
        return "d0-2"
    if bf_c != 0 and bf_h == 0 and bf_s == 0:
        return "d0-3"
    if bf_c == 0 and is_equal(bf_h, bf_s):
        return "d1-eq"
    if bf_c == 0 and bf_h > bf_s:
        return "d1->"
    if bf_c == 0 and bf_h < bf_s:
        return "d1-<"
    if bf_h == 0 and bf_c != 0 and bf_s != 0:
        return "d2"
    if bf_s == 0 and bf_c != 0 and bf_h != 0:
        return "d3"
    if is_equal(bf_h, bf_s) and (bf_c == bf_h or bf_c == bf_s or bf_c > bf_h or bf_c > bf_s):
        return "s1"
    if bf_h < bf_s:
        return "d2"
    if bf_h > bf_s:
        return "d3"
    return "unknown"

# **クラス別BFタイプ表の作成**
df_bf_type = df_class_time_normalized.pivot(index="class", columns="time_category", values=numeric_columns)
df_bf_type.columns = [f"{col[1]}_{col[0]}" for col in df_bf_type.columns]

# **クラス列が2列ある問題を完全修正**
df_bf_type.reset_index(drop=False, inplace=True)
df_bf_type = df_bf_type.loc[:, ~df_bf_type.columns.duplicated()]

# **BFタイプを判定**
for col in numeric_columns:
    df_bf_type[col] = df_bf_type.apply(lambda row: determine_bf_type(row.get(f"本来性_{col}", 0),
                                                                     row.get(f"現状_{col}", 0),
                                                                     row.get(f"志向性_{col}", 0)), axis=1)

# **不要な時制のBFの値（B列〜AE列）を削除**
bf_type_columns = ["class"] + [col for col in numeric_columns]  # BFタイプのみ保持
df_bf_type = df_bf_type[bf_type_columns]

# **5. 修正後のデータフレームを保存**
with pd.ExcelWriter(output_file_path, engine="openpyxl", mode="w") as writer:
    df_classified_mixture.to_excel(writer, sheet_name="クラス別配合表", index=False)
    df_class_time_sum.to_excel(writer, sheet_name="クラス別配合時制別合計表", index=False)
    df_class_time_normalized.to_excel(writer, sheet_name="クラス別時制別正規化配合表", index=False)
    df_bf_type.to_excel(writer, sheet_name="クラス別BFタイプ", index=False)

# **6. ヘッダーのフォントを標準にし、罫線を削除**
wb = load_workbook(output_file_path)
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    for cell in ws[1]:  # ヘッダー行を処理
        cell.font = Font(bold=False)  # 太字を解除
        cell.border = Border()  # 罫線を削除

wb.save(output_file_path)

# **7. 修正後のエクセルファイルをダウンロード**
files.download(output_file_path)

print("修正後の output.xlsx を作成し、ダウンロードリンクを表示しました。")


Saving input.xlsx to input (14).xlsx


<ipython-input-15-1d57563dd907>:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_class_time_sum = df_classified_mixture.groupby(["class", "time_category"]).sum(numeric_only=True).reset_index()
<ipython-input-15-1d57563dd907>:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_class_time_normalized[numeric_columns] = df_class_time_normalized[numeric_columns].applymap(lambda x: 0 if x < 0.1 else x)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

修正後の output.xlsx を作成し、ダウンロードリンクを表示しました。
